In [1]:
pip install pycox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 254.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 854.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 940.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.6/138.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lifelines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.5/349.5 kB 529.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.3 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4048 sha256=80e7c28c5daf1092b099f25e193449bd8636cd845b492df748ab36265782b75a
  Stored in directory: /root/.cache/pip/wheels/9f/01/ee/1331593abb5725ff7d8c1333aee93a50a1c29d6ddda9665c9f
Successfully built autograd-gamma
Note: you may need to restart the kernel to use updated packages.


In [3]:
# from sklearn.model_selection import train_test_split
# import numpy as np
# import pandas as pd
 
# data = pd.read_csv("/kaggle/input/breast-test/res_data1.csv")
# df_train,df_test = train_test_split(data,test_size = 0.2,random_state = 42)
# print(len(df_train),len(df_test))

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("/kaggle/input/breast-test/res_data1.csv")
train, test = train_test_split(data, test_size=0.2)


train["flag"] = 1
test["flag"] = 0


data = pd.concat([train, test])


features_cat = ["hualiao","xiluoda","fangliao","neifenmi","baxiang","fufa","yuanchuzhuanyi","zuzhixue_leixing",
                "T_fenqi","fenhua_xuhao","linbajie_zhuangtai","N_fenqi","TNM_fenqi","012/3","HR","HER-2",
                "Ki67","CK56","EGFR","fenxing_xuhao","shoushu_leibie","OP4"]
features_con = ["mm","LN","CCI_score","age_score","age-CCI_score"]

df_dummy = pd.get_dummies(data[features_cat])
data = pd.concat([data, df_dummy], axis = 1)

train = data[data["flag"] == 1]
test = data[data["flag"] == 0]


features = df_dummy.columns.to_list() + features_con
train_sel = train[["OS_month", "siwang"] + features]
test_sel = test[["OS_month", "siwang"] + features]
train_sel.to_csv("/kaggle/working/data_train.csv", index = False)
test_sel.to_csv("/kaggle/working/data_test.csv", index = False)

In [5]:
import numpy as np
import pandas as pd
import h5py
import scipy
from sklearn.model_selection import KFold
from scipy import ndimage
from sklearn import metrics
from lifelines.utils import concordance_index
from pycox.evaluation import EvalSurv


from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as k
from sklearn.preprocessing import normalize
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import *
from pycox.preprocessing.label_transforms import LabTransDiscreteTime

import os
import tensorflow as tf

np.random.seed(1234)
tf.random.set_seed(2020)

df_train = pd.read_csv("/kaggle/working/data_train.csv")
df_test = pd.read_csv("/kaggle/working/data_test.csv")


get_x = lambda df: (df
                    .drop(columns=['OS_month', 'siwang'])
                    .values.astype('float32'))
x_train = get_x(df_train)
x_test = get_x(df_test)


def get_y_labels(status, time):
    ret = np.ones((status.shape[0], np.max(time) + 1))
    for i in range(status.shape[0]):
        if status[i] == 1:
            ret[i, 0:time[i] - 1 + 1] = 0
        elif status[i] == 0:
            ret[i, 0:time[i] + 1] = 0
            ret[i, time[i] + 1:] = 2        
    return ret


y_train = get_y_labels(df_train['siwang'], df_train['OS_month'])
y_test = get_y_labels(df_test['siwang'], df_test['OS_month'])

time_interval =50
time_max = np.max(df_train['OS_month'])
time_length = time_max//time_interval
y_train = y_train[:, np.arange(time_interval, time_max, time_interval)]
y_test = y_test[:, np.arange(time_interval, np.max(df_test['OS_month']), time_interval)]

y_append=np.repeat(np.reshape(y_test[:,-1],(-1,1)),repeats=y_train.shape[1]-y_test.shape[1],axis=1)
y_test=np.append(y_test,y_append,axis=1)


print ("train_set_x shape: " + str(x_train.shape))
print ("train_set_y shape: " + str(y_train.shape))
print ("test_set_x shape: " + str(x_test.shape))
print ("test_set_y shape: " + str(y_test.shape))


y_train_status = to_categorical(y_train)
y_test_status = to_categorical(y_test)


def reshape_y(y):
    dim = y.shape[1]
    ret = []
    for i in range(dim):
        ret.append(y[:, i, 0:2])
    return ret        

y_train_status = reshape_y(y_train_status)
y_test_status = reshape_y(y_test_status)
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
y_train_status_f = y_train_status + [y_train]
y_test_status_f = y_test_status + [y_test]


get_target = lambda df: (df['OS_month'].values, df['siwang'].values)
durations_train, events_train = get_target(df_train)
durations_test, events_test = get_target(df_test)


def logloss(lambda3):
    def loss(y_true, y_pred):
        mask_dead = y_true[:, 1]
        mask_alive = y_true[:, 0]
        mask_censored = 1 - (mask_alive + mask_dead)
        logloss = -1 * k.mean(mask_dead * k.log(y_pred[:, 1]) + mask_alive * k.log(y_pred[:, 0]))
        - lambda3 * k.mean(y_pred[:, 1] * mask_censored * k.log(y_pred[:, 1])) # / 39899
        return logloss
    return loss


def rankingloss(y_true, y_pred, name = None):
    ranking_loss = 0
    for i in range(time_length):
        for j in range(i + 1, time_length, 1):
            tmp = y_pred[:, i] - y_pred[:, j]
            tmp1 = tmp > 0
            tmp1 = tf.cast(tmp1, tf.float32)
            ranking_loss = ranking_loss + k.mean((tmp1 * tmp * (j - i)))
    return ranking_loss



# list_lambda3 = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# list_lambda4 = [0.01, 0.1, 1, 10, 100]
# list_lr = [0.1, 0.01, 0.001, 0.0001]
# list_batch_size = [64, 128, 256, 512, 1024]
list_lambda3=[0.9]
list_lambda4=[0.1]
list_lr=[0.001]
list_batch_size=[64]

parameters = []
for lambda3 in list_lambda3:
    for lambda4 in list_lambda4:
        for lr in list_lr:
            for batch_size in list_batch_size:
                parameters.append([lambda3, lambda4, lr, batch_size])

ssmtlr_cv_results = pd.DataFrame(parameters)
ssmtlr_cv_results["cindex"] = 0

kf = KFold(n_splits = 5)


for index in range(ssmtlr_cv_results.shape[0]):
    lambda3 = ssmtlr_cv_results.iloc[index, 0]
    lambda4 = ssmtlr_cv_results.iloc[index, 1]
    lr = ssmtlr_cv_results.iloc[index, 2]
    batch_size = ssmtlr_cv_results.iloc[index, 3]

    cindexes = []
    for train_index, test_index in kf.split(df_train):
        X_tr = x_train[train_index, ]
        X_val = x_train[test_index, ]

        Y_tr_0 = y_train[train_index, ]
        Y_val_0 = y_train[test_index, ]

        Y_tr_1 = []
        Y_val_1 = []
        for i in range(time_length):
            Y_tr_1.append(y_train_status[i][train_index])
            Y_val_1.append(y_train_status[i][test_index])
        
        Y_tr = Y_tr_1 + [Y_tr_0]
        Y_val = Y_val_1 + [Y_val_0]

        input_tensor = Input((X_tr.shape[1],))
        x = input_tensor
        x = Dense(24, activation = 'sigmoid', kernel_regularizer = L1L2(l1 = 0., l2 = 0.))(x)
        x = Dense(16, activation = 'sigmoid', kernel_regularizer = L1L2(l1 = 0., l2 = 0.))(x)
        x = Dense(6, activation = 'sigmoid', kernel_regularizer = L1L2(l1 = 0., l2 = 0.))(x)

        prepare_list = {}
        for i in range(time_length):
             prepare_list['x' + str(i)] = Dense(2, activation = 'softmax', kernel_regularizer = L1L2(l1 = 0., l2 = 0.), name = 'month_' + str(i))(x)

        xx1 = concatenate(list(prepare_list.values()))
        xx2 = Lambda(lambda x: x[:, 1::2], name = 'ranking')(xx1)

        losses = {}
        loss_weights = {}
        for i in range(time_length):
            losses['month_' + str(i)] = logloss(lambda3)
            loss_weights['month_' + str(i)] = 1
        losses['ranking'] = rankingloss
        loss_weights['ranking'] = lambda4

        model = Model(input_tensor, list(prepare_list.values()) + [xx2])
        model.compile(optimizer = Adam(lr),
                      loss = losses, # 'categorical_crossentropy',
                      loss_weights = loss_weights)
        model.fit(X_tr, Y_tr, epochs = 100, validation_data=(X_val, Y_val), 
                  batch_size = batch_size, shuffle = True, # verbose = 0,
                  callbacks = [
                  ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=0, mode='auto', min_delta=0.001, cooldown=0, min_lr=0),
                  # roc_callback(training_data=(X_train, Y_train_status),validation_data=(X_test, Y_test_status)),
                     # LearningRateScheduler(lr_schedule),
                      EarlyStopping(patience = 3)])

        y_test_status_pred = model.predict(x_test)
        pred = np.array(y_test_status_pred[0:time_length])
        pred_dead = pred[:, :, 1]

        cif1 = pd.DataFrame(pred_dead, np.arange(time_length) + 1)
        ev1 = EvalSurv(1-cif1, durations_test//time_interval, events_test == 1, censor_surv='km')
        c_index = ev1.concordance_td('antolini')
        cindexes.append(c_index)

    ssmtlr_cv_results.iloc[index, 4] = np.mean(cindexes)
    ssmtlr_cv_results.to_csv('/kaggle/working/cv.results.ssmtlr.csv', index = False)
    print(ssmtlr_cv_results.iloc[index, ].values)


ssmtlr_cv_results = pd.read_csv("/kaggle/working/cv.results.ssmtlr.csv")
print(ssmtlr_cv_results["cindex"].values.max())
ind_best = ssmtlr_cv_results["cindex"].values.argmax()

lambda3 = ssmtlr_cv_results.iloc[ind_best, 0]
lambda4 = ssmtlr_cv_results.iloc[ind_best, 1]
lr = ssmtlr_cv_results.iloc[ind_best, 2]
batch_size = ssmtlr_cv_results.iloc[ind_best, 3]


input_tensor = Input((x_train.shape[1],))
x = input_tensor
x = Dense(24, activation = 'sigmoid', kernel_regularizer = L1L2(l1 = 0., l2 = 0.))(x)
x = Dense(16, activation = 'sigmoid', kernel_regularizer = L1L2(l1 = 0., l2 = 0.))(x)
x = Dense(6, activation = 'sigmoid', kernel_regularizer = L1L2(l1 = 0., l2 = 0.))(x)

prepare_list = {}
for i in range(time_length):
     prepare_list['x' + str(i)] = Dense(2, activation = 'softmax', kernel_regularizer = L1L2(l1 = 0., l2 = 0.), name = 'month_' + str(i))(x)

xx1 = concatenate(list(prepare_list.values()))
xx2 = Lambda(lambda x: x[:, 1::2], name = 'ranking')(xx1)

losses = {}
loss_weights = {}
for i in range(time_length):
    losses['month_' + str(i)] = logloss(lambda3)
    loss_weights['month_' + str(i)] = 1
losses['ranking'] = rankingloss
loss_weights['ranking'] = lambda4

model = Model(input_tensor, list(prepare_list.values()) + [xx2])
model.compile(optimizer = Adam(lr),
              loss = losses,
              loss_weights = loss_weights)

model.fit(x_train, y_train_status_f, epochs = 100, validation_data=(x_test, y_test_status_f), 
          batch_size = batch_size, shuffle = True, verbose = 0,
          callbacks = [
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=0, mode='auto', min_delta=0.001, cooldown=0, min_lr=0),
          EarlyStopping(patience = 3)])

y_test_status_pred = model.predict(x_test)
pred = np.array(y_test_status_pred[0:time_length])
pred_dead = pred[:, :, 1]



cif1 = pd.DataFrame(pred_dead, np.arange(time_length) + 1)
ev1 = EvalSurv(1-cif1, durations_test//time_interval, events_test == 1, censor_surv='km')
c_index = ev1.concordance_td('antolini')
ibs = ev1.integrated_brier_score(np.arange(time_length))
print('C-index: {:.4f}'.format(c_index))
print('IBS: {:.4f}'.format(ibs))
print(lambda3,lambda4,lr,batch_size)

train_set_x shape: (1116, 49)
train_set_y shape: (1116, 5)
test_set_x shape: (280, 49)
test_set_y shape: (280, 5)


2022-07-09 11:52:06.862048: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-07-09 11:52:07.063995: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
14/14 [==============================] - 4s 73ms/step - loss: 0.7074 - month_0_loss: 0.1993 - month_1_loss: 0.1830 - month_2_loss: 0.1111 - month_3_loss: 0.0576 - month_4_loss: 0.0372 - ranking_loss: 1.1922 - val_loss: 0.7972 - val_month_0_loss: 0.2283 - val_month_1_loss: 0.1927 - val_month_2_loss: 0.1560 - val_month_3_loss: 0.0796 - val_month_4_loss: 0.0534 - val_ranking_loss: 0.8721
Epoch 2/100
14/14 [==============================] - 0s 9ms/step - loss: 0.6205 - month_0_loss: 0.1968 - month_1_loss: 0.1632 - month_2_loss: 0.1080 - month_3_loss: 0.0544 - month_4_loss: 0.0336 - ranking_loss: 0.6458 - val_loss: 0.7260 - val_month_0_loss: 0.2267 - val_month_1_loss: 0.1764 - val_month_2_loss: 0.1518 - val_month_3_loss: 0.0750 - val_month_4_loss: 0.0493 - val_ranking_loss: 0.4670
Epoch 3/100
14/14 [==============================] - 0s 9ms/step - loss: 0.5676 - month_0_loss: 0.1957 - month_1_loss: 0.1485 - month_2_loss: 0.1049 - month_3_loss: 0.0513 - month_4_loss: 0.0316 - rank